In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import regularizers, Sequential, layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
import numpy as np
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import random

In [3]:
df = pd.read_csv('../speech_emotion_reco/data/merged_dataset.csv')
df['savee'] = df['path'].apply(lambda x: 1 if 'savee' in x else 0)
df['crema'] = df['path'].apply(lambda x: 1 if 'crema' in x else 0)
df['ravdess'] = df['path'].apply(lambda x: 1 if 'ravdess' in x else 0)
df['tess'] = df['path'].apply(lambda x: 1 if 'tess' in x else 0)

In [4]:
df.emotion.value_counts()

happy       1924
sad         1923
fear        1923
disgust     1923
angry       1923
neutral     1895
surprise     452
unknown      200
Name: emotion, dtype: int64

In [5]:
df[df['emotion']=='unknown']

,Unnamed: 0,gender,emotion,path,duration,savee,crema,ravdess,tess
3520,3520,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,1.821451,0,0,0,1
3521,3521,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,2.187574,0,0,0,1
3522,3522,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,2.157234,0,0,0,1
3523,3523,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,1.719410,0,0,0,1
3524,3524,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,2.153878,0,0,0,1
...,...,...,...,...,...,...,...,...,...
3715,3715,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,2.135329,0,0,0,1
3716,3716,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,1.759320,0,0,0,1
3717,3717,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,2.007211,0,0,0,1
3718,3718,female,unknown,../speech_emotion_reco/data/tess/YAF_pleasant_...,1.815147,0,0,0,1


In [6]:
def load_data():
    data_path = '../speech_emotion_reco/data/images_v2/'
    classes = {'happy': 0, 
               'sad': 1,
               'fear': 2,
               'disgust': 3,
               'angry': 4,
               'neutral': 5,
               'surprise': 6,
               'unknown': 7}
    imgs = []
    labels = []
    images_path = [os.path.join(data_path, path) for path in os.listdir(data_path) if path.find('.png') > 0]
    index = 0
    for path in images_path:
        if classes[df.loc[index, 'emotion']] in [0,1,2,3,4,5]:
            image = Image.open(path).convert('RGB')
            resized_image = image.resize((256,256))
            imgs.append(np.array(resized_image))
            labels.append(classes[df.loc[index, 'emotion']])
        index += 1
        
    X = np.array(imgs)
    num_classes = len(set(labels))
    y = to_categorical(labels, num_classes)

    return X, y

In [7]:
X, y = load_data()

In [8]:
X.shape

(11511, 256, 256, 3)

In [9]:
y.shape

(11511, 6)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size= 0.16, test_size=0.04, random_state=1)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [12]:
def load_model():
    model = Sequential()
    model.add(layers.Conv2D(120, kernel_size=11, activation="relu", strides=4, input_shape=(256,256,3)))
    model.add(layers.MaxPooling2D(3, strides=2))
    model.add(layers.Conv2D(256, kernel_size=5, activation="relu", strides=1))
    model.add(layers.Conv2D(384, kernel_size=3, activation="relu"))
    model.add(layers.Flatten())
    model.add(layers.Dense(2048, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(2048, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(6, activation='softmax'))
    
    return model

In [13]:
def load_simple_model():
    model = Sequential()
    model.add(layers.Conv2D(120, kernel_size=3, activation="relu", strides=1, input_shape=(256,256,3)))
    model.add(layers.MaxPooling2D(3, strides=2))
    model.add(layers.Conv2D(256, kernel_size=3, activation="relu", strides=1))
    model.add(layers.Conv2D(384, kernel_size=3, activation="relu"))
    model.add(layers.Flatten())
    model.add(layers.Dense(2048, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(2048, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(6, activation='softmax'))
    
    return model

In [14]:
def load_very_simple_model():
    model = Sequential()
    model.add(layers.Conv2D(120, kernel_size=3, activation="relu", strides=4, input_shape=(256,256,3)))
    model.add(layers.Flatten())
    model.add(layers.Dense(2048, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(6, activation='softmax'))
    
    return model

In [15]:
def build_model():  
    model = load_very_simple_model()
    opt = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [16]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_model()

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    epochs=50, 
                    batch_size=16, 
                    shuffle=False,
                    callbacks=[es],
                    verbose=2)

Metal device set to: Apple M1


2021-08-28 14:36:02.240744: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-08-28 14:36:02.242009: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-08-28 14:36:12.013939: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-28 14:36:12.020804: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/50


2021-08-28 14:36:12.704269: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

In [17]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 792992688990526326
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 312572637905083350
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
]


2021-08-28 14:43:18.713748: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-08-28 14:43:18.715188: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [22]:
from tensorflow.config import list_physical_devices
print("Num GPUs Available: ", len(list_physical_devices('GPU')))

Num GPUs Available:  1


In [24]:
from keras import backend as K
K._get_available_gpus()

['/device:GPU:0']

In [25]:
import keras
import tensorflow as tf


config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 1} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

AttributeError: module 'tensorflow' has no attribute 'ConfigProto'